In [ ]:
# ====================================================================
# CELL 1: Install Required Packages
# ====================================================================
!pip install docling openai pandas sqlalchemy pandasql python-dotenv langchain langchain-openai \
    langchain-community chromadb sentence-transformers rank-bm25 pypdf python-docx python-pptx -q


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 25.4 M

In [ ]:

print("✅ All packages installed successfully!")

# ====================================================================
# CELL 2: Import Libraries and Configuration
# ====================================================================
import os
import re
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Tuple, Optional
from collections import defaultdict
from dataclasses import dataclass
from enum import Enum
from pathlib import Path

# Docling for document processing
from docling.document_converter import DocumentConverter

# SQL Components
from sqlalchemy import create_engine, text
from pandasql import sqldf

# LangChain Components
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document, SystemMessage, HumanMessage, AIMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.tools import Tool

# Retrieval Components
from sentence_transformers import CrossEncoder
from rank_bm25 import BM25Okapi

# Azure OpenAI Configuration
os.environ["AZURE_OPENAI_API_KEY"] = "EZFRKMlQvMguHWxd4ymVjQdb5JVds9OhN5PXD3zS3dWIhfFCBpk0JQQJ99BIACL93NaXJ3w3AAAAACOGovff"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://studynet-ai-agent.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2025-01-01-preview"
os.environ["CHAT_MODEL_DEPLOYMENT"] = "chat-heavy"
os.environ["EMBEDDING_MODEL_DEPLOYMENT"] = "embed-large"

print("✅ Configuration loaded successfully!")

✅ All packages installed successfully!
✅ Configuration loaded successfully!


In [ ]:
# ====================================================================
# CELL 3: Initialize Azure OpenAI Models
# ====================================================================
# Chat Model
llm = AzureChatOpenAI(
    azure_deployment=os.environ["CHAT_MODEL_DEPLOYMENT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    temperature=0.7
)

# Embedding Model
embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.environ["EMBEDDING_MODEL_DEPLOYMENT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"]
)

# Reranker Model
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

print("✅ All models initialized successfully!")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ All models initialized successfully!


In [ ]:
# ====================================================================
# CELL 4: Data Source Classification
# ====================================================================
class DataSourceType(Enum):
    PDF_DOCS = "pdf_documents"          # PDFs, DOCX, PPTX - use RAG
    STRUCTURED_CSV = "structured_csv"   # CSV files - use SQL
    UNKNOWN = "unknown"

@dataclass
class DataSource:
    filename: str
    source_type: DataSourceType
    data: Any
    metadata: Dict[str, Any]

def classify_file(filename: str) -> DataSourceType:
    """Classify file type to determine processing strategy"""
    ext = Path(filename).suffix.lower()

    if ext in ['.pdf', '.docx', '.pptx', '.html', '.htm', '.txt', '.md']:
        return DataSourceType.PDF_DOCS
    elif ext == '.csv':
        return DataSourceType.STRUCTURED_CSV
    else:
        return DataSourceType.UNKNOWN

print("✅ Data source classifier created!")

✅ Data source classifier created!


In [ ]:
# ====================================================================
# CELL 5: Upload Files (PDF, DOCX, PPTX, CSV)
# ====================================================================
from google.colab import files

print("📁 Please upload your files (PDF, DOCX, PPTX, CSV, HTML)...")
uploaded = files.upload()

# Classify and organize uploaded files
pdf_files = []      # For RAG processing
csv_files = []      # For SQL processing
dataframes = {}     # Store CSV data
data_sources = []   # Track all sources

for filename in uploaded.keys():
    file_type = classify_file(filename)

    if file_type == DataSourceType.PDF_DOCS:
        pdf_files.append(filename)
        data_sources.append(DataSource(
            filename=filename,
            source_type=file_type,
            data=None,
            metadata={'type': 'document'}
        ))
        print(f"📄 Document: {filename}")

    elif file_type == DataSourceType.STRUCTURED_CSV:
        # Load CSV for SQL
        for encoding in ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']:
            try:
                df = pd.read_csv(filename, encoding=encoding)

                # Clean table name
                table_name = filename.replace('.csv', '').replace(' ', '_')
                table_name = re.sub(r'^[\d_]+', '', table_name)
                table_name = re.sub(r'[^\w]', '_', table_name)

                if not table_name or not table_name[0].isalpha():
                    table_name = f"table_{len(dataframes) + 1}"

                dataframes[table_name] = df
                csv_files.append(filename)

                data_sources.append(DataSource(
                    filename=filename,
                    source_type=file_type,
                    data=df,
                    metadata={'table_name': table_name, 'type': 'structured'}
                ))

                print(f"📊 CSV Table: {filename} → '{table_name}' ({df.shape[0]}×{df.shape[1]})")
                break
            except:
                continue

print(f"\n✅ Total files loaded:")
print(f"   • Documents (RAG): {len(pdf_files)}")
print(f"   • CSV Tables (SQL): {len(csv_files)}")

📁 Please upload your files (PDF, DOCX, PPTX, CSV, HTML)...


Saving 05_SN_Fees(Fees).csv to 05_SN_Fees(Fees).csv
Saving Leads Management Guide.pdf to Leads Management Guide.pdf
Saving FAQ's.pdf to FAQ's.pdf
Saving Application Management.pdf to Application Management.pdf
Saving CRM Overview.pdf to CRM Overview.pdf
📊 CSV Table: 05_SN_Fees(Fees).csv → 'SN_Fees_Fees_' (4281×12)
📄 Document: Leads Management Guide.pdf
📄 Document: FAQ's.pdf
📄 Document: Application Management.pdf
📄 Document: CRM Overview.pdf

✅ Total files loaded:
   • Documents (RAG): 4
   • CSV Tables (SQL): 1


In [ ]:
# ====================================================================
# CELL 6: Docling Document Processing for RAG
# ====================================================================
print("\n🔄 Processing documents with Docling...")

# Initialize Docling converter
converter = DocumentConverter()

document_texts = []
document_metadata = []

for pdf_file in pdf_files:
    try:
        print(f"\n  📝 Processing: {pdf_file}")

        # Convert document using Docling
        result = converter.convert(pdf_file)

        # Export to markdown for better structure preservation
        markdown_text = result.document.export_to_markdown()

        document_texts.append(markdown_text)
        document_metadata.append({
            'source': pdf_file,
            'doc_type': Path(pdf_file).suffix[1:],
            'pages': len(result.document.pages) if hasattr(result.document, 'pages') else 'unknown',
            'processing': 'docling'
        })

        print(f"  ✅ Extracted {len(markdown_text)} characters")

    except Exception as e:
        print(f"  ⚠️ Error processing {pdf_file}: {str(e)[:100]}")
        print(f"  ⏭️ Skipping this file...")

print(f"\n✅ Successfully processed {len(document_texts)} documents")


🔄 Processing documents with Docling...

  📝 Processing: Leads Management Guide.pdf


  ✅ Extracted 5808 characters

  📝 Processing: FAQ's.pdf
  ✅ Extracted 5343 characters

  📝 Processing: Application Management.pdf
  ✅ Extracted 57963 characters

  📝 Processing: CRM Overview.pdf
  ✅ Extracted 32903 characters

✅ Successfully processed 4 documents


In [ ]:
# ====================================================================
# CELL 7: Hierarchical Chunking for RAG
# ====================================================================
class HierarchicalChunker:
    """Creates parent and child chunks for better context and precision"""

    def __init__(self, parent_size=1500, child_size=500, overlap=100):
        self.parent_splitter = RecursiveCharacterTextSplitter(
            chunk_size=parent_size,
            chunk_overlap=overlap,
            separators=["\n\n", "\n", ". ", " ", ""]
        )
        self.child_splitter = RecursiveCharacterTextSplitter(
            chunk_size=child_size,
            chunk_overlap=50,
            separators=["\n\n", "\n", ". ", " ", ""]
        )

    def create_hierarchical_documents(self, text: str, metadata: Dict) -> List[Document]:
        """Create parent and child chunks maintaining semantic coherence"""
        documents = []

        # Create parent chunks
        parent_chunks = self.parent_splitter.create_documents([text])

        for p_idx, parent in enumerate(parent_chunks):
            parent_id = f"{metadata['source']}_parent_{p_idx}"

            # Add parent document
            parent.metadata.update({
                **metadata,
                'chunk_type': 'parent',
                'parent_id': parent_id,
                'chunk_index': p_idx,
                'total_parents': len(parent_chunks)
            })
            documents.append(parent)

            # Create child chunks from this parent
            child_chunks = self.child_splitter.create_documents([parent.page_content])

            for c_idx, child in enumerate(child_chunks):
                child.metadata.update({
                    **metadata,
                    'chunk_type': 'child',
                    'parent_id': parent_id,
                    'child_id': f"{parent_id}_child_{c_idx}",
                    'child_index': c_idx,
                    'total_children': len(child_chunks)
                })
                documents.append(child)

        return documents

# Create chunker
chunker = HierarchicalChunker()

# Process all documents
all_rag_documents = []

print("\n🔄 Creating hierarchical chunks...")
for text, meta in zip(document_texts, document_metadata):
    docs = chunker.create_hierarchical_documents(text, meta)
    all_rag_documents.extend(docs)

    parent_count = sum(1 for d in docs if d.metadata['chunk_type'] == 'parent')
    child_count = sum(1 for d in docs if d.metadata['chunk_type'] == 'child')

    print(f"  {meta['source']}: {parent_count} parents, {child_count} children")

print(f"\n✅ Total RAG documents: {len(all_rag_documents)}")


🔄 Creating hierarchical chunks...
  Leads Management Guide.pdf: 5 parents, 20 children
  FAQ's.pdf: 4 parents, 15 children
  Application Management.pdf: 52 parents, 182 children
  CRM Overview.pdf: 28 parents, 104 children

✅ Total RAG documents: 410


# **RAG Pipeline**

In [ ]:
# ====================================================================
# CELL 8: Query Enhancement Module
# ====================================================================
class QueryEnhancer:
    """Enhances queries with reformulations, acronym expansion, variations"""

    def __init__(self, llm):
        self.llm = llm
        self.common_acronyms = {
            'ai': 'artificial intelligence',
            'ml': 'machine learning',
            'nlp': 'natural language processing',
            'api': 'application programming interface',
            'ui': 'user interface',
            'ux': 'user experience',
            'roi': 'return on investment',
            'kpi': 'key performance indicator'
        }

    def expand_acronyms(self, query: str) -> str:
        """Expand common acronyms in query"""
        expanded = query
        for acronym, full_form in self.common_acronyms.items():
            pattern = r'\b' + acronym + r'\b'
            expanded = re.sub(pattern, f"{acronym} ({full_form})", expanded, flags=re.IGNORECASE)
        return expanded

    def remove_stop_words(self, query: str) -> str:
        """Remove common stop words for better retrieval"""
        stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for'}
        words = query.split()
        filtered = [w for w in words if w.lower() not in stop_words or len(words) <= 3]
        return ' '.join(filtered)

    def generate_query_variations(self, query: str) -> List[str]:
        """Generate multiple variations of the query using LLM"""
        prompt = f"""Generate 3 different variations of this query to improve search results.
Keep the core meaning but rephrase for better retrieval.

Original Query: "{query}"

Provide exactly 3 variations:
1. More specific/detailed version
2. Simplified/broader version
3. Alternative phrasing with synonyms

Return as JSON array: ["variation1", "variation2", "variation3"]"""

        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            import json
            variations = json.loads(response.content)
            return [query] + variations[:3]  # Include original
        except:
            # Fallback: simple variations
            return [
                query,
                self.expand_acronyms(query),
                self.remove_stop_words(query)
            ]

    def enhance_query(self, query: str) -> Dict[str, Any]:
        """Complete query enhancement pipeline"""
        # Expand acronyms
        expanded_query = self.expand_acronyms(query)

        # Generate variations
        variations = self.generate_query_variations(expanded_query)

        # Remove duplicates while preserving order
        seen = set()
        unique_variations = []
        for v in variations:
            if v.lower() not in seen:
                seen.add(v.lower())
                unique_variations.append(v)

        return {
            'original': query,
            'expanded': expanded_query,
            'variations': unique_variations
        }

query_enhancer = QueryEnhancer(llm)
print("✅ Query enhancer created!")


✅ Query enhancer created!


In [ ]:
# ====================================================================
# CELL 9: Advanced Hybrid Retrieval System
# ====================================================================
class HybridRetriever:
    """Combines semantic search with keyword-based BM25"""

    def __init__(self, vectorstore, documents, embeddings):
        self.vectorstore = vectorstore
        self.documents = documents
        self.embeddings = embeddings

        # Prepare BM25 for keyword search
        print("  🔄 Initializing BM25 for keyword search...")
        tokenized_docs = [doc.page_content.lower().split() for doc in documents]
        self.bm25 = BM25Okapi(tokenized_docs)

        print("  ✅ Hybrid retriever initialized!")

    def semantic_search(self, query: str, k: int = 10) -> List[Document]:
        """Semantic similarity search using embeddings"""
        results = self.vectorstore.similarity_search(query, k=k)
        return results

    def keyword_search(self, query: str, k: int = 10) -> List[Document]:
        """BM25 keyword-based search"""
        tokenized_query = query.lower().split()
        scores = self.bm25.get_scores(tokenized_query)

        # Get top k indices
        top_indices = np.argsort(scores)[-k:][::-1]

        return [self.documents[i] for i in top_indices if scores[i] > 0]

    def hybrid_search(self, query: str, k: int = 10, alpha: float = 0.6) -> List[Document]:
        """
        Hybrid search combining semantic and keyword
        alpha: weight for semantic (1-alpha for keyword)
        """
        # Get results from both methods
        semantic_results = self.semantic_search(query, k=k*2)
        keyword_results = self.keyword_search(query, k=k*2)

        # Score and combine using reciprocal rank fusion
        doc_scores = defaultdict(float)

        # Score semantic results
        for i, doc in enumerate(semantic_results):
            doc_id = id(doc)
            doc_scores[doc_id] += alpha * (1.0 / (i + 1))

        # Score keyword results
        for i, doc in enumerate(keyword_results):
            doc_id = id(doc)
            doc_scores[doc_id] += (1 - alpha) * (1.0 / (i + 1))

        # Sort by combined score
        sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

        # Return top k unique documents
        seen_content = set()
        results = []
        for doc_id, score in sorted_docs:
            doc = next((d for d in semantic_results + keyword_results if id(d) == doc_id), None)
            if doc and doc.page_content not in seen_content:
                seen_content.add(doc.page_content)
                results.append(doc)
                if len(results) >= k:
                    break

        return results

    def multi_query_retrieval(self, queries: List[str], k: int = 10) -> List[Document]:
        """Retrieve using multiple query variations and deduplicate"""
        all_results = []
        seen_content = set()

        for query in queries:
            results = self.hybrid_search(query, k=k)
            for doc in results:
                if doc.page_content not in seen_content:
                    seen_content.add(doc.page_content)
                    all_results.append(doc)

        return all_results[:k*2]  # Return more for reranking

print("✅ Hybrid retriever class created!")





✅ Hybrid retriever class created!


In [ ]:
# ====================================================================
# CELL 10: Intelligent Reranker
# ====================================================================
class IntelligentReranker:
    """LLM-based semantic reranking with relevance scoring"""

    def __init__(self, llm, cross_encoder):
        self.llm = llm
        self.cross_encoder = cross_encoder

    def cross_encoder_rerank(self, query: str, documents: List[Document], top_k: int = 5) -> List[Tuple[Document, float]]:
        """Fast reranking using cross-encoder"""
        if not documents:
            return []

        pairs = [[query, doc.page_content[:512]] for doc in documents]  # Limit length
        scores = self.cross_encoder.predict(pairs)

        # Sort by score
        doc_scores = list(zip(documents, scores))
        doc_scores.sort(key=lambda x: x[1], reverse=True)

        return doc_scores[:top_k]

    def llm_rerank(self, query: str, documents: List[Document], top_k: int = 3) -> List[Tuple[Document, float]]:
        """Deep semantic reranking using LLM with context awareness"""
        if not documents:
            return []

        # Prepare documents for evaluation
        doc_texts = "\n\n".join([
            f"Document {i+1}:\n{doc.page_content[:500]}..."
            for i, doc in enumerate(documents[:10])
        ])

        prompt = f"""Rank these documents by relevance to the query. Consider semantic meaning and context.

Query: "{query}"

{doc_texts}

Return document numbers (1-based) in order of relevance as JSON array: [3, 1, 5, ...]"""

        try:
            response = self.llm.invoke([HumanMessage(content=prompt)])
            import json
            rankings = json.loads(response.content)

            reranked = []
            for rank_idx, doc_num in enumerate(rankings[:top_k]):
                if 1 <= doc_num <= len(documents):
                    reranked.append((documents[doc_num-1], 1.0 / (rank_idx + 1)))

            return reranked
        except:
            return self.cross_encoder_rerank(query, documents, top_k)

    def rerank(self, query: str, documents: List[Document], method: str = "cross_encoder", top_k: int = 5) -> List[Document]:
        """Main reranking interface with relevance scoring"""
        if method == "llm":
            ranked = self.llm_rerank(query, documents, top_k)
        else:
            ranked = self.cross_encoder_rerank(query, documents, top_k)

        return [doc for doc, score in ranked]

reranker_system = IntelligentReranker(llm, reranker)
print("✅ Intelligent reranker created!")


✅ Intelligent reranker created!


In [ ]:
# ====================================================================
# CELL 11: Create Vector Store with Rate Limiting
# ====================================================================
import time
from tqdm import tqdm

print("\n🔄 Creating vector store for documents...")

if len(all_rag_documents) > 0:
    vectorstore = Chroma(
        embedding_function=embeddings,
        persist_directory="./chroma_db"
    )

    # Batch processing with rate limiting
    BATCH_SIZE = 10
    DELAY = 3

    batches = [all_rag_documents[i:i + BATCH_SIZE] for i in range(0, len(all_rag_documents), BATCH_SIZE)]

    successful = 0
    failed = 0

    for batch_idx, batch in enumerate(tqdm(batches, desc="Creating embeddings")):
        try:
            vectorstore.add_documents(batch)
            successful += len(batch)

            if batch_idx < len(batches) - 1:
                time.sleep(DELAY)

        except Exception as e:
            if "429" in str(e):
                print(f"\n⏳ Rate limit - waiting 60s...")
                time.sleep(60)
                try:
                    vectorstore.add_documents(batch)
                    successful += len(batch)
                except:
                    failed += len(batch)
            else:
                failed += len(batch)

    vectorstore.persist()

    print(f"\n✅ Vector store created!")
    print(f"   • Successful: {successful}")
    if failed > 0:
        print(f"   • Failed: {failed}")

    # Initialize hybrid retriever
    hybrid_retriever = HybridRetriever(vectorstore, all_rag_documents, embeddings)
else:
    print("⚠️ No documents to process for RAG")
    vectorstore = None
    hybrid_retriever = None



🔄 Creating vector store for documents...


/tmp/ipython-input-1081579279.py:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
Creating embeddings: 100%|██████████| 41/41 [02:25<00:00,  3.54s/it]


✅ Vector store created!
   • Successful: 410
  🔄 Initializing BM25 for keyword search...
  ✅ Hybrid retriever initialized!



/tmp/ipython-input-1081579279.py:44: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
# ====================================================================
# CELL 12: Conversation Memory Manager
# ====================================================================
class ConversationMemory:
    """Manages conversation context with token-aware summarization"""

    def __init__(self, llm, max_tokens: int = 2000):
        self.llm = llm
        self.max_tokens = max_tokens
        self.conversations = {}  # Session-based isolation

    def get_session(self, session_id: str = "default") -> List:
        """Get or create conversation history for session"""
        if session_id not in self.conversations:
            self.conversations[session_id] = []
        return self.conversations[session_id]

    def add_message(self, session_id: str, role: str, content: str):
        """Add message to conversation history"""
        history = self.get_session(session_id)

        if role == "user":
            history.append(HumanMessage(content=content))
        elif role == "assistant":
            history.append(AIMessage(content=content))
        else:
            history.append(SystemMessage(content=content))

        # Check if we need to summarize
        if self._estimate_tokens(history) > self.max_tokens:
            self._summarize_history(session_id)

    def _estimate_tokens(self, messages: List) -> int:
        """Rough token estimation"""
        total_chars = sum(len(str(msg.content)) for msg in messages)
        return total_chars // 4

    def _summarize_history(self, session_id: str):
        """Summarize old conversation to preserve tokens"""
        history = self.get_session(session_id)

        if len(history) <= 4:
            return

        old_messages = history[:-4]
        recent_messages = history[-4:]

        old_text = "\n".join([f"{msg.__class__.__name__}: {msg.content}" for msg in old_messages])

        summary_prompt = f"""Summarize this conversation history concisely:

{old_text}

Provide a brief summary (2-3 sentences):"""

        try:
            response = self.llm.invoke([HumanMessage(content=summary_prompt)])
            summary = response.content

            self.conversations[session_id] = [
                SystemMessage(content=f"Previous conversation: {summary}")
            ] + recent_messages
        except:
            self.conversations[session_id] = history[-6:]

    def get_context(self, session_id: str, max_messages: int = 10) -> List:
        """Get recent conversation context"""
        history = self.get_session(session_id)
        return history[-max_messages:]

    def clear_session(self, session_id: str):
        """Clear session history"""
        if session_id in self.conversations:
            del self.conversations[session_id]

memory_manager = ConversationMemory(llm)
print("✅ Conversation memory manager created!")

✅ Conversation memory manager created!


In [ ]:
# ====================================================================
# CELL 13: SQL Agent Tools
# ====================================================================
def execute_sql_query(query: str) -> str:
    """Execute SQL query on CSV dataframes"""
    try:
        query = query.strip()
        if query.startswith("```sql"):
            query = query.replace("```sql", "").replace("```", "").strip()

        print(f"\n🔍 Executing SQL: {query}\n")

        result = sqldf(query, dataframes)

        if result is None or result.empty:
            return "Query returned no results."

        result_str = f"Query returned {len(result)} rows:\n\n"
        result_str += result.to_string(index=False, max_rows=50)

        if len(result) > 50:
            result_str += f"\n\n(showing first 50 of {len(result)} rows)"

        return result_str

    except Exception as e:
        return f"❌ SQL Error: {str(e)}\nAvailable tables: {', '.join(dataframes.keys())}"

def get_sql_schema_info(table_name: str = "") -> str:
    """Get SQL database schema"""
    if table_name and table_name in dataframes:
        df = dataframes[table_name]
        info = f"# Table: {table_name}\n\n**Rows:** {len(df)}\n**Columns:** {len(df.columns)}\n\n"

        for col in df.columns:
            info += f"- {col} ({df[col].dtype}): "
            if df[col].dtype in ['int64', 'float64']:
                info += f"range {df[col].min()}-{df[col].max()}\n"
            else:
                top = df[col].value_counts().head(3)
                info += f"top values: {', '.join(map(str, top.index))}\n"
        return info
    else:
        schema = "# Available SQL Tables:\n\n"
        for tname, df in dataframes.items():
            schema += f"## {tname}\n- Rows: {len(df)}\n- Columns: {', '.join(df.columns)}\n\n"
        return schema if dataframes else "No SQL tables available."

print("✅ SQL tools created!")

✅ SQL tools created!


In [ ]:


# ====================================================================
# CELL 14: RAG Retrieval Tool
# ====================================================================
def rag_search(query: str, top_k: int = 5) -> str:
    """Advanced RAG search with enhancement and reranking"""

    if not hybrid_retriever:
        return "❌ No documents available for RAG search. Please upload PDF/DOCX/PPTX files."

    try:
        # Enhance query
        enhanced = query_enhancer.enhance_query(query)

        print(f"\n🔍 RAG Search")
        print(f"Original: {enhanced['original']}")
        print(f"Variations: {len(enhanced['variations'])}")

        # Multi-query retrieval
        retrieved_docs = hybrid_retriever.multi_query_retrieval(
            enhanced['variations'],
            k=top_k
        )

        print(f"Retrieved: {len(retrieved_docs)} documents")

        # Rerank
        reranked_docs = reranker_system.rerank(
            query,
            retrieved_docs,
            method="cross_encoder",
            top_k=top_k
        )

        print(f"Reranked to: {len(reranked_docs)} documents\n")

        if not reranked_docs:
            return "No relevant documents found."

        result = f"Found {len(reranked_docs)} relevant documents:\n\n"

        for i, doc in enumerate(reranked_docs, 1):
            result += f"--- Document {i} ---\n"
            result += f"Source: {doc.metadata.get('source', 'Unknown')}\n"
            result += f"Type: {doc.metadata.get('chunk_type', 'Unknown')}\n"
            result += f"Content:\n{doc.page_content[:500]}...\n\n"

        return result

    except Exception as e:
        return f"❌ RAG Error: {str(e)}"

print("✅ RAG search tool created!")


✅ RAG search tool created!


In [ ]:

# ====================================================================
# CELL 15: Query Type Classification
# ====================================================================
class QueryType(Enum):
    STRUCTURED_SQL = "structured_sql"
    SEMANTIC_RAG = "semantic_rag"
    HYBRID = "hybrid"

@dataclass
class QueryAnalysis:
    query_type: QueryType
    confidence: float
    reasoning: str
    requires_sql: bool
    requires_rag: bool

def classify_query_type(query: str) -> QueryAnalysis:
    """Intelligently classify query to route to SQL, RAG, or both"""

    classification_prompt = f"""Analyze this query and determine the best approach.

Query: "{query}"

Available Data Sources:
- SQL Tables (CSV): {', '.join(dataframes.keys()) if dataframes else 'None'}
- Documents (RAG): {len(pdf_files)} files (PDF/DOCX/PPTX)

Classification:
- **STRUCTURED_SQL**: Exact filtering, aggregations, counts, calculations on CSV data
- **SEMANTIC_RAG**: Conceptual questions, explanations, document content search
- **HYBRID**: Needs both structured data AND document understanding

Respond in JSON:
{{
  "query_type": "structured_sql" | "semantic_rag" | "hybrid",
  "confidence": 0.0-1.0,
  "reasoning": "brief explanation"
}}"""

    try:
        response = llm.invoke([HumanMessage(content=classification_prompt)])
        import json
        result = json.loads(response.content)
        query_type = QueryType(result['query_type'])

        return QueryAnalysis(
            query_type=query_type,
            confidence=result['confidence'],
            reasoning=result['reasoning'],
            requires_sql=query_type in [QueryType.STRUCTURED_SQL, QueryType.HYBRID],
            requires_rag=query_type in [QueryType.SEMANTIC_RAG, QueryType.HYBRID]
        )
    except:
        # Fallback classification
        sql_keywords = ['count', 'sum', 'average', 'total', 'filter', 'group', 'table']
        rag_keywords = ['what', 'explain', 'how', 'why', 'describe', 'document']

        has_sql = any(kw in query.lower() for kw in sql_keywords) and len(dataframes) > 0
        has_rag = any(kw in query.lower() for kw in rag_keywords) and len(pdf_files) > 0

        if has_sql and has_rag:
            qtype = QueryType.HYBRID
        elif has_sql:
            qtype = QueryType.STRUCTURED_SQL
        else:
            qtype = QueryType.SEMANTIC_RAG

        return QueryAnalysis(
            query_type=qtype,
            confidence=0.7,
            reasoning="Fallback classification",
            requires_sql=has_sql,
            requires_rag=has_rag
        )

print("✅ Query classifier created!")

✅ Query classifier created!


In [ ]:


# ====================================================================
# CELL 16: Master Orchestrator Agent
# ====================================================================
class MasterOrchestrator:
    """Intelligently routes queries to SQL, RAG, or both"""

    def __init__(self, llm, memory_manager):
        self.llm = llm
        self.memory = memory_manager

        # Define tools
        self.tools = []

        # Add SQL tools if available
        if len(dataframes) > 0:
            self.tools.extend([
                Tool(
                    name="sql_query",
                    func=execute_sql_query,
                    description="""Execute SQL queries on CSV data for structured retrieval.
                    Use for: filtering, aggregations, counts, sums, averages, grouping, sorting.
                    Input: Valid SQL query string."""
                ),
                Tool(
                    name="get_sql_schema",
                    func=get_sql_schema_info,
                    description="""Get SQL database schema and table information.
                    Use to understand CSV data structure before writing queries.
                    Input: Optional table name."""
                )
            ])

        # Add RAG tool if available
        if len(pdf_files) > 0:
            self.tools.append(
                Tool(
                    name="rag_search",
                    func=rag_search,
                    description="""Semantic search across PDF/DOCX/PPTX documents.
                    Use for: explanations, definitions, document content, qualitative analysis.
                    Input: Natural language query string."""
                )
            )

        # Create agent
        self.agent = self._create_agent()

    def _create_agent(self):
        """Create the orchestrator agent"""

        # Build capabilities description
        capabilities = []
        if len(dataframes) > 0:
            capabilities.append(f"1. **SQL Database**: {len(dataframes)} CSV tables with structured data")
        if len(pdf_files) > 0:
            capabilities.append(f"2. **Document Search (RAG)**: {len(pdf_files)} documents (PDF/DOCX/PPTX)")

        capabilities_text = "\n".join(capabilities) if capabilities else "No data sources available"

        system_prompt = f"""You are an intelligent data assistant with access to multiple data sources.

**Available Data Sources:**
{capabilities_text}

**Your Tools:**
{"- sql_query: Execute SQL on CSV tables" if len(dataframes) > 0 else ""}
{"- get_sql_schema: View table structures" if len(dataframes) > 0 else ""}
{"- rag_search: Search documents semantically" if len(pdf_files) > 0 else ""}

**Decision Making:**
1. Analyze what the user needs
2. Determine which data source(s) to use:
   - SQL for precise data queries (counts, sums, filtering)
   - RAG for document understanding (explanations, concepts)
   - Both for comprehensive analysis
3. Use appropriate tools
4. Synthesize results clearly

**Best Practices:**
- Check schema before SQL queries
- Use RAG for document-based questions
- Combine results when using both sources
- Explain your reasoning
- Be conversational and helpful

Always provide accurate, well-explained answers."""

        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="chat_history", optional=True),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

        agent = create_openai_functions_agent(self.llm, self.tools, prompt)

        return AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,
            max_iterations=8,
            handle_parsing_errors=True
        )

    def query(self, question: str, session_id: str = "default") -> str:
        """Process query with intelligent routing"""

        # Classify query
        classification = classify_query_type(question)

        print(f"\n{'='*70}")
        print(f"🤔 Query Analysis:")
        print(f"  Type: {classification.query_type.value}")
        print(f"  Confidence: {classification.confidence:.2f}")
        print(f"  Reasoning: {classification.reasoning}")
        print(f"  Needs SQL: {classification.requires_sql}")
        print(f"  Needs RAG: {classification.requires_rag}")
        print(f"{'='*70}\n")

        # Add to memory
        self.memory.add_message(session_id, "user", question)

        # Get context
        context = self.memory.get_context(session_id)

        # Build input
        input_data = {
            "input": question,
            "chat_history": context[:-1]
        }

        try:
            # Execute agent
            response = self.agent.invoke(input_data)
            answer = response['output']

            # Add to memory
            self.memory.add_message(session_id, "assistant", answer)

            print(f"\n{'='*70}")
            print("💡 ANSWER:")
            print(f"{'='*70}")
            print(answer)
            print(f"{'='*70}\n")

            return answer

        except Exception as e:
            error_msg = f"❌ Error: {str(e)}"
            print(error_msg)
            return error_msg

# Initialize orchestrator
orchestrator = MasterOrchestrator(llm, memory_manager)
print("✅ Master Orchestrator created!")

✅ Master Orchestrator created!


In [ ]:

# ====================================================================
# CELL 17: Main Query Interface
# ====================================================================
def query(question: str, session_id: str = "default") -> str:
    """
    Main interface for querying the hybrid system.
    Automatically routes to SQL, RAG, or both.

    Args:
        question: Natural language question
        session_id: Session ID for conversation continuity

    Returns:
        Answer string
    """
    return orchestrator.query(question, session_id)

print("✅ Query interface ready!")
print("\n🎯 Example Usage:")
print('query("What tables do I have?")')
print('query("Show me the top 5 providers by revenue")')
print('query("What does the document say about policies?")')
print('query("Compare revenue data and explain the trends")')

✅ Query interface ready!

🎯 Example Usage:
query("What tables do I have?")
query("Show me the top 5 providers by revenue")
query("What does the document say about policies?")
query("Compare revenue data and explain the trends")


In [ ]:



# ====================================================================
# CELL 18: Interactive Mode
# ====================================================================
def interactive_mode(session_id: str = "default"):
    """Interactive chat interface with conversation memory"""

    print("\n" + "="*70)
    print("🤖 HYBRID RAG + SQL AGENT - INTERACTIVE MODE")
    print("="*70)
    print(f"Session ID: {session_id}")
    print(f"\nData Sources:")
    print(f"  • SQL Tables: {len(dataframes)}")
    print(f"  • Documents: {len(pdf_files)}")
    print(f"\nCommands:")
    print("  'quit' - Exit")
    print("  'clear' - Clear history")
    print("  'history' - Show history")
    print("  'sources' - Show data sources")
    print("="*70 + "\n")

    while True:
        question = input("❓ You: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Goodbye!")
            break

        if question.lower() == 'clear':
            memory_manager.clear_session(session_id)
            print("✅ History cleared.\n")
            continue

        if question.lower() == 'history':
            history = memory_manager.get_context(session_id)
            print("\n📜 Conversation History:")
            for msg in history:
                role = msg.__class__.__name__.replace("Message", "")
                print(f"  {role}: {msg.content[:100]}...")
            print()
            continue

        if question.lower() == 'sources':
            print("\n📊 Available Data Sources:")
            print(f"\nSQL Tables ({len(dataframes)}):")
            for name, df in dataframes.items():
                print(f"  • {name}: {df.shape[0]} rows × {df.shape[1]} cols")
            print(f"\nDocuments ({len(pdf_files)}):")
            for doc in pdf_files:
                print(f"  • {doc}")
            print()
            continue

        if not question:
            continue

        try:
            query(question, session_id)
        except Exception as e:
            print(f"❌ Error: {str(e)}\n")

print("✅ Interactive mode ready!")
print("💡 Run: interactive_mode()")

✅ Interactive mode ready!
💡 Run: interactive_mode()


In [ ]:
interactive_mode()


🤖 HYBRID RAG + SQL AGENT - INTERACTIVE MODE
Session ID: default

Data Sources:
  • SQL Tables: 1
  • Documents: 4

Commands:
  'quit' - Exit
  'clear' - Clear history
  'history' - Show history
  'sources' - Show data sources

❓ You: my name is kabid yeiad, im 23years of age. suggestme some it courses under $20k in sydney

🤔 Query Analysis:
  Type: structured_sql
  Confidence: 0.95
  Reasoning: The user is requesting a list of IT courses available in Sydney with fees under $20,000. This is a factual, filter-based query that can be answered by performing exact filtering on the SN_Fees_Fees_ SQL table (CSV data). There is no need for conceptual document search or semantic reasoning.
  Needs SQL: True
  Needs RAG: False



> Entering new AgentExecutor chain...

Invoking: `get_sql_schema` with ``


# Available SQL Tables:

## SN_Fees_Fees_
- Rows: 4281
- Columns: ProviderID, ProviderName, CourseName, COURSEID, YEAR, UNIT, UNITPRICE, UNITCOUNT, CURRENCY, TOTALANNUALFEE, TOTALCOURSEFEE, Fee

In [ ]:


# ====================================================================
# CELL 19: Analytics & Reporting Functions
# ====================================================================
def analyze_data_sources():
    """Comprehensive analysis of all data sources"""

    print("\n" + "="*70)
    print("📊 DATA SOURCE ANALYSIS")
    print("="*70)

    # SQL Tables
    if len(dataframes) > 0:
        print(f"\n🗄️ SQL TABLES ({len(dataframes)}):")
        for table_name, df in dataframes.items():
            print(f"\n  Table: {table_name}")
            print(f"  Shape: {df.shape[0]} rows × {df.shape[1]} cols")
            print(f"  Columns: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
            print(f"  Memory: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")

            # Missing values
            missing = df.isnull().sum().sum()
            if missing > 0:
                print(f"  Missing: {missing} values")
    else:
        print("\n🗄️ No SQL tables loaded")

    # Documents
    if len(pdf_files) > 0:
        print(f"\n📄 DOCUMENTS ({len(pdf_files)}):")
        for doc in pdf_files:
            doc_type = Path(doc).suffix[1:].upper()
            print(f"  • {doc} ({doc_type})")

        print(f"\n📚 RAG STATISTICS:")
        print(f"  Total Chunks: {len(all_rag_documents)}")

        parent_count = sum(1 for d in all_rag_documents if d.metadata.get('chunk_type') == 'parent')
        child_count = sum(1 for d in all_rag_documents if d.metadata.get('chunk_type') == 'child')

        print(f"  Parent Chunks: {parent_count}")
        print(f"  Child Chunks: {child_count}")
    else:
        print("\n📄 No documents loaded")

    print("\n" + "="*70)

def export_sql_results(sql_query: str, filename: str = "results.csv"):
    """Execute SQL and export to CSV"""
    try:
        result = sqldf(sql_query, dataframes)
        result.to_csv(filename, index=False)
        print(f"✅ Exported to {filename} ({len(result)} rows)")
        return result
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

def generate_report():
    """Generate comprehensive system report"""

    report = []
    report.append("="*70)
    report.append("HYBRID RAG + SQL SYSTEM REPORT")
    report.append("="*70)
    report.append(f"\nGenerated: {pd.Timestamp.now()}")

    report.append(f"\n\nDATA SOURCES:")
    report.append(f"  SQL Tables: {len(dataframes)}")
    report.append(f"  Documents: {len(pdf_files)}")

    if dataframes:
        report.append(f"\n\nSQL TABLES:")
        for name, df in dataframes.items():
            report.append(f"  {name}: {df.shape[0]}×{df.shape[1]}")

    if pdf_files:
        report.append(f"\n\nDOCUMENTS:")
        for doc in pdf_files:
            report.append(f"  {doc}")

    report.append(f"\n\nRAG SYSTEM:")
    report.append(f"  Total Chunks: {len(all_rag_documents)}")
    report.append(f"  Vector Store: {'Active' if vectorstore else 'Inactive'}")

    report_text = "\n".join(report)
    print(report_text)

    with open("system_report.txt", "w") as f:
        f.write(report_text)

    print("\n✅ Report saved to system_report.txt")
    return report_text

print("✅ Analytics functions ready!")

In [ ]:


# ====================================================================
# CELL 20: System Summary & Quick Start
# ====================================================================
def show_usage_guide():
    """Display comprehensive usage guide"""

    guide = """
╔══════════════════════════════════════════════════════════════════════╗
║           HYBRID RAG + SQL SYSTEM - USAGE GUIDE                      ║
╚══════════════════════════════════════════════════════════════════════╝

🎯 QUERY EXAMPLES:

📊 SQL Queries (CSV Data):
  • "How many rows in each table?"
  • "Show me top 10 providers by revenue"
  • "Calculate average fees"
  • "Filter records where amount > 1000"
  • "Group by category and count"

📄 RAG Queries (Documents):
  • "What does the document say about X?"
  • "Explain the policy mentioned in the PDF"
  • "Summarize the key points"
  • "Find information about Y"
  • "What are the requirements?"

🔄 Hybrid Queries (Both):
  • "Show revenue data and explain the trends"
  • "Compare providers and explain differences"
  • "Analyze the data and provide insights from documents"
  • "What do the documents say about [table column]?"

💡 FEATURES:

✓ Hierarchical Chunking (1500/500 chars)
✓ Query Enhancement (variations, acronyms)
✓ Hybrid Retrieval (semantic + keyword)
✓ Intelligent Reranking (cross-encoder)
✓ Conversation Memory (auto-summarization)
✓ Automatic Query Routing
✓ Multi-turn Conversations
✓ Session Management

🚀 QUICK START:

1. Interactive Mode:
   interactive_mode()

2. Single Queries:
   query("your question here")

3. Check Data:
   analyze_data_sources()

4. Generate Report:
   generate_report()

═══════════════════════════════════════════════════════════════════════
"""
    print(guide)

# Display system status
print("\n" + "="*70)
print("🚀 SYSTEM INITIALIZED - READY FOR QUERIES")
print("="*70)

print(f"\n📊 Loaded Data:")
print(f"  • SQL Tables: {len(dataframes)}")
for name in dataframes.keys():
    print(f"    - {name}")
print(f"  • Documents: {len(pdf_files)}")
for doc in pdf_files:
    print(f"    - {doc}")

print(f"\n🎯 Available Functions:")
print("  • query('question')           - Ask anything")
print("  • interactive_mode()          - Chat interface")
print("  • analyze_data_sources()      - Data analysis")
print("  • generate_report()           - System report")
print("  • show_usage_guide()          - Full guide")

print("\n💡 Start with: interactive_mode() or show_usage_guide()")
print("="*70)

show_usage_guide()